In [10]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
import json

In [11]:
def zero_based_mapping(data) :
    with open('/opt/ml/movie-recommendation/data/train/zero_mapping.json', 'r') as f:
        dict_data= json.load(f)

    n_user = len(dict_data['user'])
    n_item = len(dict_data['item'])

    data['user']  = data['user'].map(lambda x : dict_data['user'][str(x)])
    data['item']  = data['item'].map(lambda x : dict_data['item'][str(x)])
    
    return data, n_user, n_item

In [6]:
data_dir = '/opt/ml/movie-recommendation/data/'
train_df = pd.read_csv(os.path.join(data_dir+'train', 'train_ratings.csv')) # 전체 학습 데이터

In [12]:
train_df,_,_ =zero_based_mapping(train_df)

In [7]:
# Extract Top Most Popular movies
top_ratio = 0.1

num_items = train_df['item'].nunique()
num_users = train_df['user'].nunique()
most_popular_items = train_df['item'].value_counts()[:int(num_items*top_ratio)].index.values

most_popular_items = set(most_popular_items)

In [14]:
total_items = set(train_df['item'])
observed_items_per_user = list(train_df.groupby('user')['item'])

In [22]:
train_dir = '/opt/ml/movie-recommendation/data/'
train_df = pd.read_csv(os.path.join(train_dir+'train', 'train_ratings.csv'))
total_items = set(train_df['item'])
observed_items_per_user = list(train_df.groupby('user')['item'])
unobserved_dict = dict()
for user, observed_items in observed_items_per_user:
    unobserved_dict[user] = list(total_items - set(observed_items))

100%|██████████| 31360/31360 [00:13<00:00, 2305.35it/s]


In [6]:
# 각 유저마다 안본 영화만 선택
unseen_items_dfs = list()

for user, observed_items in tqdm(observed_items_per_user):
    observed_items = set(observed_items)
    unseen_item = list(most_popular_items - observed_items)

    user_id = [user]*len(unseen_item)
    unseen_items_dfs.append(pd.DataFrame(zip(user_id,unseen_item), columns=['user','item']))

test_df = pd.concat(unseen_items_dfs, axis = 0, sort=False)

100%|██████████| 31360/31360 [00:30<00:00, 1031.37it/s]


In [7]:
test_df = test_df.sort_values(by=['user'])
test_df.reset_index(drop=True, inplace=True)
test_df

,user,item
0,11,2
1,11,3362
2,11,7451
3,11,79132
4,11,1307
...,...,...
18182955,138493,2700
18182956,138493,648
18182957,138493,2692
18182958,138493,2683


In [9]:
test_df.to_csv(data_dir+'eval/Unobserved Cases Top 10%.csv')

In [13]:
# Extract Top Most Popular movies
num_items = train_df['item'].nunique()
num_users = train_df['user'].nunique()

items = set(train_df['item'])
observed_items_per_user = list(train_df.groupby('user')['item'])

# 각 유저마다 안본 영화만 선택
unseen_items_dfs = list()

for user, observed_items in tqdm(observed_items_per_user):
    observed_items = set(observed_items)
    unseen_item = list(items - observed_items)

    user_id = [user]*len(unseen_item)
    unseen_items_dfs.append(pd.DataFrame(zip(user_id,unseen_item), columns=['user','item']))

test_df = pd.concat(unseen_items_dfs, axis = 0, sort=False)

test_df = test_df.sort_values(by=['user'])
test_df.reset_index(drop=True, inplace=True)
test_df

100%|██████████| 31360/31360 [02:32<00:00, 206.03it/s]


,user,item
0,11,2
1,11,7177
2,11,89090
3,11,7171
4,11,89087
...,...,...
208313044,138493,101739
208313045,138493,3435
208313046,138493,85354
208313047,138493,3448
